https://huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/components.ipynb

In [1]:
# !pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface llama-index-llms-mlx -U -q

In [2]:
!pip show llama_index

Name: llama-index
Version: 0.12.25
Summary: Interface between LLMs and your data
Home-page: 
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /Users/ljoana/.pyenv/versions/3.11.9/envs/blog/lib/python3.11/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [3]:
import os
from pathlib import Path

import chromadb
import llama_index
import nest_asyncio
from datasets import load_dataset
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings
from llama_index.llms.mlx import MLXLLM
from llama_index.vector_stores.chroma import ChromaVectorStore

# Persona Database

In [4]:
# https://huggingface.co/datasets/dvilasuero/finepersonas-v0.1-tiny
dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")
dataset

Dataset({
    features: ['id', 'persona', 'model_name_embeddings', 'embedding', 'nn_indices', 'nn_scores', 'projection', 'cluster_label', 'summary_label'],
    num_rows: 5000
})

In [5]:
Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

A local art historian and museum professional interested in 19th-century American art and the local cultural heritage of Cincinnati.

# Embedding Persona Documents

In [6]:
reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

In [7]:
documents[0]

Document(id_='2fee10c1-8eab-4919-bd51-bee5882f2f3c', embedding=None, metadata={'file_path': '/Users/ljoana/repos/LLMs-Journey/Agents/data/persona_0.txt', 'file_name': 'persona_0.txt', 'file_type': 'text/plain', 'file_size': 132, 'creation_date': '2025-03-23', 'last_modified_date': '2025-03-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='A local art historian and museum professional interested in 19th-century American art and the local cultural heritage of Cincinnati.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

In [8]:
print(documents[0])

Doc ID: 2fee10c1-8eab-4919-bd51-bee5882f2f3c
Text: A local art historian and museum professional interested in
19th-century American art and the local cultural heritage of
Cincinnati.


In [9]:
print(documents[1])

Doc ID: 622b8d1e-23cf-4fa2-bb1f-4b78f9336105
Text: An anthropologist or a cultural expert interested in the
intricacies of Cypriot culture, history, and society, particularly
someone who has spent considerable time researching and living in
Cyprus to gain a deep understanding of its people, customs, and way of
life.


In [10]:
# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

/var/folders/r0/6h61qg4157lcdfpct0nslmd40000gn/T/ipykernel_5954/1609717970.py:5: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5"),


In [11]:
# run the pipeline sync or async
nodes = await pipeline.arun(documents=documents[:10])
nodes

[TextNode(id_='746d778d-07e6-4b90-9374-24c4d5124380', embedding=[-0.016962038353085518, -0.020101305097341537, 0.04712051898241043, 0.015091117471456528, -0.040820129215717316, -0.03728194534778595, 0.0088089220225811, 0.024028290063142776, -0.06148263439536095, -0.06373955309391022, -0.0088106794282794, -0.018149483948946, -0.05040455982089043, 0.030572209507226944, -0.009642815217375755, -0.042164720594882965, -0.002408028347417712, 0.08752823621034622, -0.025158006697893143, 0.013464724645018578, 0.0028630681335926056, -0.06087138503789902, 0.051802366971969604, -0.0029338342137634754, -0.024022871628403664, 0.002772703068330884, 0.011880548670887947, 0.007090989034622908, -0.026497501879930496, -0.12555910646915436, -0.053310196846723557, 0.018436705693602562, -0.012514184229075909, 0.01582437753677368, 0.05625845119357109, 0.027687447145581245, -0.0024652755819261074, 0.03413606807589531, -0.00011809895659098402, 0.005119895096868277, 0.033376920968294144, 0.032458651810884476, -0

# Storing and Indexing Documents

In [12]:
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [13]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

/var/folders/r0/6h61qg4157lcdfpct0nslmd40000gn/T/ipykernel_5954/344299432.py:4: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5"),


In [14]:
nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

10

In [15]:
nodes

[TextNode(id_='62f2387b-3f45-4b93-bf44-7371d8ecbadc', embedding=[-0.016962038353085518, -0.020101305097341537, 0.04712051898241043, 0.015091117471456528, -0.040820129215717316, -0.03728194534778595, 0.0088089220225811, 0.024028290063142776, -0.06148263439536095, -0.06373955309391022, -0.0088106794282794, -0.018149483948946, -0.05040455982089043, 0.030572209507226944, -0.009642815217375755, -0.042164720594882965, -0.002408028347417712, 0.08752823621034622, -0.025158006697893143, 0.013464724645018578, 0.0028630681335926056, -0.06087138503789902, 0.051802366971969604, -0.0029338342137634754, -0.024022871628403664, 0.002772703068330884, 0.011880548670887947, 0.007090989034622908, -0.026497501879930496, -0.12555910646915436, -0.053310196846723557, 0.018436705693602562, -0.012514184229075909, 0.01582437753677368, 0.05625845119357109, 0.027687447145581245, -0.0024652755819261074, 0.03413606807589531, -0.00011809895659098402, 0.005119895096868277, 0.033376920968294144, 0.032458651810884476, -0

In [16]:
embed_model = HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

/var/folders/r0/6h61qg4157lcdfpct0nslmd40000gn/T/ipykernel_5954/2267681564.py:1: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  embed_model = HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5")


# Querying the Index

In [17]:
nest_asyncio.apply()  # This is needed to run the query engine
llm = MLXLLM(model_name="mlx-community/Qwen2.5-Coder-7B-Instruct-4bit")
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [18]:
response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)
response

Response(response='I am an anthropologist specializing in Cypriot culture, history, and society. My research has taken me to Cyprus, where I have lived and conducted extensive studies to understand the people, customs, and way of life. My work aims to capture the essence of Cypriot society and its rich heritage, offering insights that are both academic and accessible to a broader audience. Through my travels and research, I have gained a deep appreciation for the complexities and nuances of Cypriot culture, making me a valuable resource for those interested in this fascinating region.<|im_end|>\n<|im_start|><|im_end|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_end|>\n', source_nodes=[NodeWithScore(node=TextNode(id_='44c42c59-9286-4a22-a30d-73a9ff73bd2c', embedding=None, meta

In [19]:
response.response

'I am an anthropologist specializing in Cypriot culture, history, and society. My research has taken me to Cyprus, where I have lived and conducted extensive studies to understand the people, customs, and way of life. My work aims to capture the essence of Cypriot society and its rich heritage, offering insights that are both academic and accessible to a broader audience. Through my travels and research, I have gained a deep appreciation for the complexities and nuances of Cypriot culture, making me a valuable resource for those interested in this fascinating region.<|im_end|>\n<|im_start|><|im_end|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_start|>\n<|im_end|>\n'

# Evaluation and Observability

In [20]:
# query index
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

True